In [8]:
import os,json,csv, re
import opencc

txt_file = 'article.txt'
csv_file = 'article.csv'

# converter = opencc.OpenCC('s2twp.json')

In [9]:
# read .txt

rows = []
with open(txt_file,'r',encoding='utf-8') as f:
    rows = f.readlines()
    for i in range(len(rows)):
        try:
            rows[i] = json.loads(rows[i])
        except:
            pass

In [ ]:
# data cleaning
for i in range(len(rows)):
    try:
        text = rows[i]['content']
        text = re.sub(pattern="[&].*?[;]",repl='',string=text)  
        text = re.sub(pattern="[<u>].*?[</u>]",repl='',string=text)
        text = re.sub(pattern="[<].*?[>]",repl='',string=text)  
        text = re.sub(pattern="[https://].*?[/]",repl='',string=text)  
        text = text.replace('作文/Www.Z','')
        text = text.replace('\n','')

        # text = converter.convert(text)
        rows[i]['content'] = text
    except:
        pass

In [ ]:
# save to csv
labels = list(rows[0].keys())

with open(csv_file, 'w') as f:
    writer = csv.DictWriter(f, fieldnames=labels)
    writer.writeheader()
    for row in rows:
        try:
            writer.writerow(row)
        except:
            pass
